In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from SD import stochastic_descent

from quantum_state import i
from quantum_state import i, evolution_from_protocol, spectral_time_evolution, compute_fidelity
from qutip import basis, Bloch
from gif import qutip_qstate
import os
import pandas as pd

In [ ]:
def correlation(matrix, h):
    '''
    Given a matrix this function computes the correlation quantity Q(T) as in the paper.
    Matrix must have dimension (n_protocols,lenght_of_protocol)    
    '''
    n_row = matrix.shape[0]
    n_col = matrix.shape[1] 
    
    mean_hx = np.array([matrix[:,i].mean() for i in range(n_col)]) #mean over all protocols at fixed time
    avg_over_h = np.array([np.array([ (matrix[i,j]-mean_hx[j])**2 for i in range(n_row)]).sum()/n_row for j in range(n_col)])
    avg_over_Nt = (1/((h*h)*n_col))*(avg_over_h.sum())
    return  avg_over_Nt

In [ ]:
#Starting qstates and their normalization
qstart = np.array([-1/2 - (np.sqrt(5))/2 ,1], dtype=complex)
qtarget = np.array([+1/2 + (np.sqrt(5))/2 ,1], dtype=complex)
qstart=qstart/np.sqrt(np.vdot(qstart,qstart))
qtarget=qtarget/np.sqrt(np.vdot(qtarget,qtarget))

#SD parameters
L = 1
h = 4
h_list = [-h,h]
dt = 0.01
iterations = int(1e3)
percentage_flips = 7
exp_decay = False
metropolis = False

print("-------------PARAMETERS for SD---------------")
print("Number of qubits (L):", L)
print("Magnetic fields(h):", h)
print("Timesteps (dt):", dt)
print("Iterations:", iterations)
print("Percentage of flips:", percentage_flips)
print("Exp decays of flips:", exp_decay)
print("Metropolis:", metropolis)
print("\n")
print("\n")

#parameters for Fig. pag 2 ---- Calculation of fidelity and Q(T)
step_in_time_grid = 0.25
times = np.arange(0,4.25,step=step_in_time_grid)
iter_for_each_time = 12

fidelity_for_txt = []
print("--------------PARAMETERS for Plotting-------------")
print("Timegrid:", times)
print("Repetition at each timestep:", iter_for_each_time)
print("\n")

params_dict = {"L":L, "h":h, "dt":dt, "iterations":iterations, "percentage_flips":percentage_flips, 
               "exp_decay":exp_decay, "metropolis":metropolis, "times":times, "iter_for_each_time": iter_for_each_time}
params_df = pd.DataFrame.from_dict(params_dict, orient="index")

In [ ]:
#UNCOMMENT TO RUN
print("WARNING! Attenzione a non sovrascrivere")

#save run parameters and date in custom named folder
custom_name_dir = "data"
os.system("mkdir "+custom_name_dir)
os.system("mkdir "+custom_name_dir+"/protocols")
params_df.to_csv(custom_name_dir+"/parameters.csv")

intermediete_result = True

for T in times:
    temp_fid = []
    best_prot = []
    for _ in range(iter_for_each_time):
        best_protocol, fidelity = stochastic_descent(qstart, qtarget, L, T, dt, iterations, 
                        flips = percentage_flips, field_list = h_list,beta=1,exp_decay_flip=exp_decay,
                                             metropolis_choice=metropolis,verbose = True, check_norm = False)
        temp_fid.append(fidelity[-1]) #at fixed T we will have "iter_for_each_time" evaluations of fidelity
        best_prot.append(best_protocol) #all iter_for_each_time best protocols are stored in this variable 
                                        #and saved when new rounf over T starts
    fidelity_for_txt.append(temp_fid) #fidelity evaluations are stored in the same "fidelity_fot_txt"
                                      #variable that will have dimension len(times)*iter_for_each_time
    best_prot = np.array([best_prot])
    with open(custom_name_dir +'/protocols/testT'+str(T)+'.npy', 'wb') as f:
        np.save(f,best_prot)
    f.close()
    
    if intermediete_result and T !=0: #if T = 0 q cannot be computed
        data = np.load(custom_name_dir + "/protocols/testT"+str(T)+".npy")[0,:,:] #first dimension is redundant 
        print("Q value is {} at T = {}", .format{correlation(data, h), T})
        
#fidelity values are saved at the end
np.savetxt(custom_name_dir + '/fidelity.txt', fidelity_for_txt, delimiter = ',',header="Matrix with as entries the values of fidelity dimension times x iterations")


In [ ]:
#PLOT 

q=[0]
for T in times[1:]:
    data = np.load("protocols/testT"+str(t)+".npy")[0,:,:] #first dimension is redundant 
    #!warning: correlation normalization term musb be changed according to field values
    q.append(correlation(data))


loaded_fidelity = pd.read_csv('fidelity.txt', skiprows=1,header=None)

#loaded_fidelity = loaded_fidelity.loc[(loaded_fidelity.index % 19 == 0) & (loaded_fidelity.index > 0)]

fig, ax = plt.subplots(figsize=(10,7))
#plot Fidelity values
ax.errorbar(times,loaded_fidelity.mean(axis=1).values, yerr=loaded_fidelity.std(axis=1).values, color="r")
ax.scatter(times,loaded_fidelity.mean(axis=1).values,color='r',label="Fidelity")
#plot Q values
ax.plot(times, q, marker="o", color="goldenrod", markersize=7, label="q(T)")

ax.set_title(r"Phase Diagram"+"\n"+"Iterations for Each Point = "+str(iter_for_each_time), fontsize=18)
ax.set_xlabel("T [a.u.]", fontsize=18)
#ax.set_ylabel(r" $Fidelity$", fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.legend(fontsize=13, loc=5)
fig.tight_layout()
plt.show()